# Fillomino

1. Enter a number in each cell of the grid. Cells with the same number form regions which must contain exactly as many cells as the number indicates.
2. Two different regions of the same size must not be orthogonally adjacent (diagonally is allowed).
3. Two clue numbers may belong to the same region; some regions may not contain any clue number.

In [31]:
def readGrid(path):
    with open(f"../assets/data/Fillomino/problems/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res
    
m, n, res = readGrid("12_5x5")

In [33]:
from collections import deque
from typing import Tuple, List, Set, FrozenSet
from itertools import combinations

class Fillomino:
    def __init__(self, m: int, n: int, grid: List[List[str]]):
        self.m = m
        self.n = n
        self.grid = grid
        self.fixed = [[False for _ in range(n)] for _ in range(m)]
        self.uncertain_char = ".*-"
    
    def get_connected_block(self, i: int, j: int) -> Tuple[int, Set[Tuple[int, int]]]:
        """BFS获取连通块及其相邻空白格"""
        num = self.grid[i][j]
        queue = deque([(i, j)])
        visited = set([(i, j)])
        blank_neighbors = set()
        
        while queue:
            x, y = queue.popleft()
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.m and 0 <= ny < self.n:
                    if (nx, ny) not in visited:
                        if self.grid[nx][ny] == num:
                            visited.add((nx, ny))
                            queue.append((nx, ny))
                        elif self.grid[nx][ny] in self.uncertain_char:
                            blank_neighbors.add((nx, ny))
        
        return len(visited), blank_neighbors

    def apply_single_exit_strategy(self): 
        iteration = 0
        # Apply Strategy until no more changes
        while self.single_exit_strategy():
            iteration += 1
        
        print(f"\nSingle Exit Strategy Fills {iteration} times.")
        
    def single_exit_strategy(self) -> bool:
        """实施单出口策略，返回是否进行了任何填充"""
        changed = False
        
        for i in range(self.m):
            for j in range(self.n):
                if self.grid[i][j] not in self.uncertain_char:
                    curr_size, blank_neighbors = self.get_connected_block(i, j)
                    target = int(self.grid[i][j])
                    
                    if curr_size < target and len(blank_neighbors) == 1:
                        x, y = blank_neighbors.pop()
                        self.grid[x][y] = self.grid[i][j]
                        self.fixed[x][y] = True
                        print(f"[SES] Fill ({x}, {y}) -> {self.grid[i][j]}")
                        changed = True
                        # 填充后立即返回，以便重新评估网格
                        return changed
        
        return changed

    def get_neighbors(self, cell: Tuple[int, int]) -> List[Tuple[int, int]]:
        """获取单元格的相邻单元格（正交方向）"""
        i, j = cell
        neighbors = []
        for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < self.m and 0 <= nj < self.n:
                neighbors.append((ni, nj))
        return neighbors

    def get_start_cells(self, group: Set[Tuple[int, int]]) -> Set[Tuple[int, int]]:
        """获取有相邻空白格的起始单元格"""
        start_cells = set()
        for cell in group:
            for neighbor in self.get_neighbors(cell):
                if self.grid[neighbor[0]][neighbor[1]] in self.uncertain_char:
                    start_cells.add(cell)
                    break
        return start_cells

    def bfs_expansion(self, start: Tuple[int, int], target_size: int, 
                     current_group: Set[Tuple[int, int]]) -> List[Set[Tuple[int, int]]]:
        """BFS扩展找到所有可能的完成路径"""
        num = self.grid[start[0]][start[1]]
        paths = []
        
        # 使用BFS找到所有可能的扩展路径
        # 初始状态：当前组和已扩展的单元格
        queue = deque()
        initial_state = (current_group.copy(), set())  # (当前组, 已扩展的单元格)
        queue.append(initial_state)
        
        while queue:
            current_group_state, expanded = queue.popleft()
            
            # 如果已经达到目标大小，保存路径
            if len(current_group_state) == target_size:
                paths.append(expanded)
                continue
            
            # 如果超过了目标大小，跳过
            if len(current_group_state) > target_size:
                continue
            
            # 找到所有可能的扩展单元格
            possible_expansions = set()
            for cell in current_group_state:
                for neighbor in self.get_neighbors(cell):
                    if (neighbor not in current_group_state and 
                        self.grid[neighbor[0]][neighbor[1]] in self.uncertain_char):
                        # 检查扩展是否有效（不违反相邻相同数字规则）
                        valid = True
                        for nn in self.get_neighbors(neighbor):
                            if (nn not in current_group_state and 
                                nn != neighbor and
                                self.grid[nn[0]][nn[1]] == num):
                                valid = False
                                break
                        
                        if valid:
                            possible_expansions.add(neighbor)
            
            # 对每个可能的扩展，创建新状态
            for expansion in possible_expansions:
                new_group = current_group_state | {expansion}
                new_expanded = expanded | {expansion}
                
                # 检查新状态是否有效
                valid_state = True
                for cell in new_group:
                    for neighbor in self.get_neighbors(cell):
                        if (neighbor not in new_group and
                            self.grid[neighbor[0]][neighbor[1]] == num):
                            valid_state = False
                            break
                    if not valid_state:
                        break
                
                if valid_state:
                    queue.append((new_group, new_expanded))
        
        return paths

    def find_structurally_forced_cells(self, group: Set[Tuple[int, int]], 
                                      target_size: int) -> Set[Tuple[int, int]]:
        """找到结构强制单元格"""
        # 如果组已经达到目标大小，返回空集
        if len(group) >= target_size:
            return set()
        
        # 获取起始单元格
        start_cells = self.get_start_cells(group)
        
        # 收集所有完成路径
        all_paths = []
        for start in start_cells:
            paths = self.bfs_expansion(start, target_size, group)
            all_paths.extend(paths)
        
        # 如果没有找到路径，返回空集
        if not all_paths:
            return set()
        
        # 找到所有路径的交集
        forced_cells = set.intersection(*[set(path) for path in all_paths]) if all_paths else set()
        
        return forced_cells

    def apply_structurally_force_strategy(self):
        """应用结构强制单元格策略"""
        changed = False
        
        # 找到所有不完整的组
        visited = set()
        for i in range(self.m):
            for j in range(self.n):
                if (i, j) not in visited and self.grid[i][j] not in self.uncertain_char:
                    # 获取当前组
                    num = self.grid[i][j]
                    target_size = int(num)
                    
                    # 使用BFS找到整个组
                    group = set()
                    queue = deque([(i, j)])
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) not in group and self.grid[x][y] == num:
                            group.add((x, y))
                            visited.add((x, y))
                            for nx, ny in self.get_neighbors((x, y)):
                                if (nx, ny) not in group and self.grid[nx][ny] == num:
                                    queue.append((nx, ny))
                    
                    # 如果组不完整，应用结构强制策略
                    if len(group) < target_size:
                        forced_cells = self.find_structurally_forced_cells(group, target_size)
                        
                        # 填充强制单元格
                        for cell in forced_cells:
                            x, y = cell
                            if self.grid[x][y] in self.uncertain_char:
                                self.grid[x][y] = num
                                self.fixed[x][y] = True
                                print(f"[SFS] Fill ({x}, {y}) -> {num}")
                                changed = True
        
        return changed

    def solve(self):
        """求解Fillomino谜题"""
        print("初始网格:")
        for row in self.grid:
            print(row)
        
        iteration = 0
        changed = True
        
        while changed:
            iteration += 1
            print(f"\n--- 第 {iteration} 次迭代 ---")
            
            changed = False
            
            # 应用单出口策略
            ses_changed = self.single_exit_strategy()
            if ses_changed:
                changed = True
                print("应用单出口策略后:")
                for row in self.grid:
                    print(row)
                continue  # 优先应用单出口策略
            
            # 应用结构强制策略
            sfs_changed = self.apply_structurally_force_strategy()
            if sfs_changed:
                changed = True
                print("应用结构强制策略后:")
                for row in self.grid:
                    print(row)
        
        print("\n最终网格:")
        for row in self.grid:
            print(row)

if __name__ == "__main__":
    # 输入网格
    input_grid = [
        ['3', '-', '-', '-', '5'],
        ['-', '3', '-', '1', '-'],
        ['1', '-', '3', '-', '2'],
        ['-', '5', '-', '3', '-'],
        ['5', '-', '1', '-', '-']
    ]
    
    m, n = 5, 5
    solver = Fillomino(m, n, input_grid)
    solver.solve()

初始网格:
['3', '-', '-', '-', '5']
['-', '3', '-', '1', '-']
['1', '-', '3', '-', '2']
['-', '5', '-', '3', '-']
['5', '-', '1', '-', '-']

--- 第 1 次迭代 ---
[SFS] Fill (0, 2) -> 5
[SFS] Fill (0, 3) -> 5
应用结构强制策略后:
['3', '-', '5', '5', '5']
['-', '3', '-', '1', '-']
['1', '-', '3', '-', '2']
['-', '5', '-', '3', '-']
['5', '-', '1', '-', '-']

--- 第 2 次迭代 ---

最终网格:
['3', '-', '5', '5', '5']
['-', '3', '-', '1', '-']
['1', '-', '3', '-', '2']
['-', '5', '-', '3', '-']
['5', '-', '1', '-', '-']
